In [1]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("Sales Data Analysis").getOrCreate()

# File location and type
file_location = "/Users/inbravo/Documents/GitHub/python-feature-set/dataset/super-market/sales-less-record.csv"

# Read the CSV data from the file
df = spark.read.format("csv") \
  .option("inferSchema", True) \
  .option("header", True) \
  .option("sep", ",") \
  .load(file_location)

# Show the first 5 rows of the DataFrame
df.show(5)

ModuleNotFoundError: No module named 'pyspark'

In [ ]:
from pyspark.sql.functions import round
from pyspark.sql.functions import sum as _sum
from pyspark.sql.types import IntegerType
   
# Typecast the 'Gross income' values to Integer type    
df = df.withColumn("Gross_Income", df["Gross_Income"].cast(IntegerType()))

# Round gross income to 2 decimal places
df = df.withColumn("Gross_Income", round(df["Gross_Income"], 2))

# Group by the "Product line"
df = df.groupBy("Product_Line")

# Sum of the gross income
gross_income_sum = _sum("Gross_Income")

# Sum the gross income
df_round = df.agg(gross_income_sum).alias("Gross_Income")

# Show the first 5 rows of the DataFrame
df_round.show(5)

In [ ]:
from pyspark.sql.functions import round
from pyspark.sql.functions import sum as _sum
from pyspark.sql.types import IntegerType

grouped_df=df.groupBy("Product_line").agg(_sum("Gross_Income").alias("Total_Gross"))   #checking to see what type of products yeilds more income
df_round = grouped_df.withColumn("Rounded_Total_Sales", round(grouped_df["Total_Gross"], 2))
display(df_round)

In [ ]:
# entire code in single place
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, hour, to_timestamp, sum, avg, dayofweek

# Initialize Spark Session (if not already created in Databricks)
spark = SparkSession.builder.appName("SupermarketSalesETL").getOrCreate()

# Load the dataset from DBFS
file_path = "/FileStore/tables/supermarket_sales___Sheet1-1.csv"
df = spark.read.option("header", True).option("inferSchema", True).csv(file_path)

# 1️⃣ Data Cleaning
df = df.withColumn("Date", to_date(col("Date"), "M/d/yyyy"))
df = df.withColumn("Time", to_timestamp(col("Time"), "HH:mm"))
df = df.withColumn("Hour", hour(col("Time")))
df = df.withColumn("Day_of_Week", dayofweek(col("Date")))

# 2️⃣ Feature Engineering & Aggregations

# Total Sales Per Branch
df_branch_sales = df.groupBy("Branch").agg(sum("Total").alias("Total_Sales"))

# Average Basket Size (Avg Total per Transaction)
df_avg_basket = df.agg(avg("Total").alias("Avg_Basket_Size"))

# Total Spending Per Customer Type & Gender
df_customer_spend = df.groupBy("Customer type", "Gender").agg(sum("Total").alias("Total_Spending"))

# Preferred Payment Methods
df_payment_method = df.groupBy("Payment").agg(sum("Total").alias("Total_Spent"))

# Peak Sales Hours
df_peak_hours = df.groupBy("Hour").agg(sum("Total").alias("Total_Sales")).orderBy(col("Total_Sales").desc())

# Busiest Days of the Week
df_busy_days = df.groupBy("Day_of_Week").agg(sum("Total").alias("Total_Sales")).orderBy(col("Total_Sales").desc())

# Total Revenue & Profit per Branch
df_branch_financials = df.groupBy("Branch").agg(
    sum("Total").alias("Total_Revenue"),
    sum("gross income").alias("Total_Profit")
)

# Average Rating Per Product Line
df_avg_rating = df.groupBy("Product line").agg(avg("Rating").alias("Avg_Rating"))

# Most Frequently Purchased Products
df_top_products = df.groupBy("Product line").agg(sum("Quantity").alias("Total_Quantity")).orderBy(col("Total_Quantity").desc())

# Showing Results, We can do the same with display for visualization
df_branch_sales.show()
df_avg_basket.show()
df_customer_spend.show()
df_payment_method.show()
df_peak_hours.show()
df_busy_days.show()
df_branch_financials.show()
df_avg_rating.show()
df_top_products.show()